Instalar librerías necesarias

In [ ]:
!pip install torch transformers sentence-transformers faiss-cpu pymupdf pdfplumber nltk sentence_transformers datasets
!pip install --upgrade accelerate
!pip install bitsandbytes accelerate
!pip install PyPDF2
!pip install pdfplumber

Importo fuente de datos

In [22]:
import pdfplumber
import re
import nltk

# Descargar recursos necesarios para NLTK
nltk.download('punkt')  # Descargar el modelo general
nltk.download('punkt_tab')  # Instalar el recurso faltante

# ✅ 1. Leer el PDF y extraer texto
pdf_path = "/content/guerra_ucrania.pdf"  # Ruta ajustada
text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Texto limpio (primeras 1000 letras):
Invasión rusa a Ucrania La invasión rusa a Ucrania, también conocida Invasión rusa a Ucrania como la guerra de Ucrania, iniciada el 24 de febrero de 2022, constituye una escalada de la guerra ruso- Parte de la guerra ruso-ucraniana ucraniana que comenzó tras los sucesos del Euromaidán en 2014. Se trata del mayor conflicto militar convencional en Europa desde la Segunda Guerra Mundial. La cifra precisa de víctimas se desconoce; se estima que hasta mediados de 2023, había causado la muerte de más de nueve mil civiles y decenas de miles de soldados. Los combates también han generado la mayor crisis de refugiados en 4 el continente desde la Segunda Guerra Mundial: más de 7,3 millones de ucranianos han abandonado el país Situación a 20 de marzo de 2025 (UTC±0) y más de 7,1 millones se han desplazado internamente.5 4 Además, la guerra ha causado daño Territorio Territorio Territorio controlado recuperado por ocupado por Rusia ambiental significativo y ha 

Limpieza de caracteres especiales y referencias

In [ ]:
# ✅ 2. Limpiar el texto:
# Eliminar URLs
text = re.sub(r'http\S+|www\S+', '', text)

# Eliminar referencias a notas y texto irrelevante como: "nota 3", "nota 4"
text = re.sub(r'nota \d+', '', text)

# Eliminar caracteres especiales que puedan estar presentes
# text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Eliminar caracteres no ASCII

# Eliminar saltos de línea innecesarios
text = text.replace('\n', ' ').replace('\r', '')

# Eliminar espacios múltiples y dejar solo uno
text = ' '.join(text.split())

# ✅ 3. Mostrar un fragmento del texto limpio
print("Texto limpio (primeras 1000 letras):")
print(text[:1000])

Tokenización utilizando nltk en español

In [ ]:

# ✅ 4. Dividir el texto en oraciones (tokenización)
try:
    sentences = nltk.sent_tokenize(text, language='spanish')  # Usar el recurso de tokenización español
except Exception as e:
    print(f"Error al tokenizar: {e}")
    sentences = []  # En caso de error, dejar la lista vacía

# ✅ 5. Guardar el texto limpio en un archivo para verificación
with open("/content/guerra_ucrania_limpio.txt", "w") as file:
    file.write(text)

# ✅ 6. Guardar las oraciones (opcional) en un archivo de texto
if sentences:  # Solo guardar si la tokenización fue exitosa
    with open("/content/guerra_ucrania_sentences.txt", "w") as file:
        for sentence in sentences:
            file.write(sentence + "\n")
else:
    print("No se pudieron generar oraciones.")

# ✅ 7. Mostrar las primeras 1000 palabras del texto limpio
cleaned_sample = ' '.join(text.split()[:1000])
print("Primeras 1000 palabras del texto limpio:")
print(cleaned_sample)

Tokenizo la informacion y genero un resumen con GPT2

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargar el modelo y el tokenizer (GPT-2)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_summary(text):
    # Crear un prompt para el resumen
    prompt = f"Resumir el siguiente texto: {text}"

    # Tokenizar el texto y generar la salida
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)  # Aumentar el max_length a 1024
    summary_ids = model.generate(
        inputs['input_ids'],
        max_new_tokens=150,  # Aumentamos la longitud de los tokens nuevos generados
        num_beams=5,         # Utilizar beam search para una mejor generación
        early_stopping=True,
        no_repeat_ngram_size=2,  # Evitar la repetición de n-gramas
        pad_token_id=tokenizer.eos_token_id,  # Especificar el id del token de padding (fin de secuencia)
        attention_mask=inputs['attention_mask']  # Asegurarse de que la máscara de atención esté presente
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Ejemplo de uso
text_to_summarize = text[:1000]  # Toma un fragmento del texto limpio para resumir
summary = generate_summary(text_to_summarize)
print("Resumen generado: ", summary)



Resumen generado:  Resumir el siguiente texto: Invasión rusa a Ucrania La invasión rusa a Ucrania, también conocida Invasión rusa a Ucrania como la guerra de Ucrania, iniciada el 24 de febrero de 2022, constituye una escalada de la guerra ruso- Parte de la guerra ruso-ucraniana ucraniana que comenzó tras los sucesos del Euromaidán en 2014. Se trata del mayor conflicto militar convencional en Europa desde la Segunda Guerra Mundial. La cifra precisa de víctimas se desconoce; se estima que hasta mediados de 2023, había causado la muerte de más de nueve mil civiles y decenas de miles de soldados. Los combates también han generado la mayor crisis de refugiados en 4 el continente desde la Segunda Guerra Mundial: más de 7,3 millones de ucranianos han abandonado el país Situación a 20 de marzo de 2025 (UTC±0) y más de 7,1 millones se han desplazado internamente.5 4 Además, la guerra ha causado daño Territorio Territorio Territorio controlado recuperado por ocupado por Rusia ambiental significa